<a href="https://colab.research.google.com/github/jaesunkkk/22/blob/main/1011_(%EC%9B%94)_%EC%84%B8%EC%85%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install surprise
import surprise

     |████████████████████████████████| 11.8 MB 6.3 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1619417 sha256=d5fea24fbdcca07678e928ca140e62a4428805e8c7b4fdf39e65344ed058dcc6
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [4]:
from surprise import SVD
from surprise import Dataset 
from surprise import accuracy 
from surprise.model_selection import train_test_split

In [5]:
data = Dataset.load_builtin('ml-100k') 
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [6]:
algo = SVD()
algo.fit(trainset)

In [7]:
predictions = algo.test( testset )
print('prediction type :',type(predictions), ' size:',len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type : <class 'list'>  size: 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.681603101628057, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=4.009825326366315, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.062746356923398, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.7342300375970066, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.462472484485326, details={'was_impossible': False})]

In [8]:
[ (pred.uid, pred.iid, pred.est) for pred in predictions[:3] ]

[('120', '282', 3.681603101628057),
 ('882', '291', 4.009825326366315),
 ('535', '507', 4.062746356923398)]

In [9]:
# 사용자 아이디, 아이템 아이디는 문자열로 입력해야 함. 
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.32   {'was_impossible': False}


In [10]:
accuracy.rmse(predictions)

RMSE: 0.9484


0.9483714596147019

In [12]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [13]:
import pandas as pd

ratings = pd.read_csv('/content/gdrive/MyDrive/essa/ratings.csv')
# ratings_noh.csv 파일로 unload 시 index 와 header를 모두 제거한 새로운 파일 생성.  
ratings.to_csv('/content/gdrive/MyDrive/essa/ratings_noh.csv', index=False, header=False)

In [15]:
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data=Dataset.load_from_file('/content/gdrive/MyDrive/essa/ratings_noh.csv',reader=reader)

In [16]:
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

# 수행시마다 동일한 결과 도출을 위해 random_state 설정 
algo = SVD(n_factors=50, random_state=0)

# 학습 데이터 세트로 학습 후 테스트 데이터 세트로 평점 예측 후 RMSE 평가
algo.fit(trainset) 
predictions = algo.test( testset )
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

In [19]:
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv('/content/gdrive/MyDrive/essa/ratings.csv') 
reader = Reader(rating_scale=(0.5, 5.0))

# ratings DataFrame 에서 컬럼은 사용자 아이디, 아이템 아이디, 평점 순서를 지켜야 합니다. 
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset) 
predictions = algo.test( testset )
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

In [20]:
from surprise.model_selection import cross_validate 

# Pandas DataFrame에서 Surprise Dataset으로 데이터 로딩 
ratings = pd.read_csv('/content/gdrive/MyDrive/essa/ratings.csv') # reading data in pandas df
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algo = SVD(random_state=0) 
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8750  0.8769  0.8775  0.8719  0.8682  0.8739  0.0035  
MAE (testset)     0.6714  0.6725  0.6755  0.6710  0.6659  0.6713  0.0031  
Fit time          4.70    4.70    4.71    4.69    4.74    4.71    0.01    
Test time         0.14    0.29    0.18    0.34    0.18    0.23    0.07    


{'fit_time': (4.704568862915039,
  4.7019736766815186,
  4.71161150932312,
  4.6909339427948,
  4.735605955123901),
 'test_mae': array([0.67138976, 0.67246958, 0.67552874, 0.6710234 , 0.66593712]),
 'test_rmse': array([0.87495934, 0.87691334, 0.87752422, 0.87194901, 0.86817774]),
 'test_time': (0.14126944541931152,
  0.28637146949768066,
  0.18303704261779785,
  0.34062886238098145,
  0.18485212326049805)}

In [21]:
from surprise.model_selection import GridSearchCV

# 최적화할 파라미터들을 딕셔너리 형태로 지정. 
param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200] }

# CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mse 로 수행 하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

# 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.876935920174348
{'n_epochs': 20, 'n_factors': 50}
